In [2]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

In [22]:
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.Many years later, the village was holding a hunting competition. The girl's father, the priest, the prince, and the advisor were all there. The girl recognized them (knew who they were), but they did not recognize her.

At the competition, the girl stood up and told her life story to the crowd.

Everyone immediately recognized her. They understood she was telling the truth! The girl, the prince, and her father were all happily reunited (brought together again). The priest and advisor were banished to prison.

They soon learned that the son who had been stolen was actually living in the village. The girl and her husband, the prince, were then reunited with their son.

The prince was so happy that he passed a law that said that hunters could not hunt (kill) gazelles. From that day on, the gazelles were protected from harm.
""".split()


vocab=set(raw_text)
vocab_size=len(vocab)
word2idx={word:i for i,word in enumerate(vocab)}
idx2word={i:word for i,word in enumerate(vocab)}
#print(word2idx)
embedding_size=6


data=[]
for i in range(2,len(raw_text)-2):
    context=[raw_text[i - 2], raw_text[i - 1],raw_text[i + 1], raw_text[i + 2]]
    target=raw_text[i]
    data.append((context,target))

print(data[0:6])

def make_context_vector(context,word2idx):
    idxs=[word2idx[w] for w in context]
    return torch.tensor(idxs,dtype=torch.long)
def make_context_vector1(context,word2idx):
    idxs=word2idx[context]
    return torch.tensor(idxs,dtype=torch.long)


a=make_context_vector(data[0][0],word2idx)
print (data[0][1]) 


e=nn.Embedding(vocab_size,embedding_size)
a=e(a)
#print(a.size())
max_n=vocab_size+2

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea'), (['the', 'idea', 'a', 'computational'], 'of')]
about


In [4]:







class LSTM(nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,output_size,num_layers):
        super(LSTM,self).__init__()
        
        self.hidden_size=hidden_size
        
        self.output_size=output_size
        self.input_size=input_size
        self.num_layers=num_layers
        self.lstm=nn.LSTM(self.input_size,self.hidden_size,self.num_layers)
        
        self.word_embeddings=nn.Embedding(vocab_size,embedding_size)
        
        self.linear=nn.Linear(self.hidden_size,embedding_size)
        
        
        
        
        
    def forward(self,inputi,target):
        
        embedsi=self.word_embeddings(inputi)
        
        embedso=self.word_embeddings(target)
        
        out,self.hidden=self.lstm(embedsi.view(len(inputi),1,-1))
        #out=torch.randn(1,hidden_size)
        
        y_pred = self.linear(out[-1].view(1, -1))
    
        loss=((embedso-y_pred)**2).mean()
    
        return loss
        


In [5]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=1):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(max_n, self.hidden_dim, self.num_layers)
        self.word_embeddings=nn.Embedding(vocab_size,embedding_size)
        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, max_n)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, inputi,target):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        embedsi=self.word_embeddings(inputi)
        embedsi=torch.nn.functional.one_hot(inputi,max_n)
        embedso=self.word_embeddings(target)
        targett=one_hot = torch.nn.functional.one_hot(target, max_n)
        
        targett=targett.type(torch.long)
        embedsi=embedsi.type(torch.float)
        out,self.hidden=self.lstm(embedsi.view(len(inputi),1,-1))
        #out=torch.randn(1,self.hidden_dim)
        
        y_pred = self.linear(out[-1].view(1, -1))

        y_pred=F.softmax(y_pred,dim=1)
        
        targett=targett.view(1,-1)
        
        return y_pred,targett

In [7]:
epochs=50
model=LSTM(max_n,max_n,1,max_n,1)
lossfunc=nn.NLLLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=0.1)
with torch.no_grad():
    inputii=make_context_vector(data[0][0],word2idx)
    target=make_context_vector1(data[0][1],word2idx)
    #print(target)
    a,b=model(inputii,target)
    print(a)
    print(b)

for iters in range(1,15):    
    for epoch in range (1,epochs):
        
        
        loss=torch.tensor([0])
        optimiser.zero_grad()
        model.zero_grad()
        for a,b in data:
            
            loss=loss.type(torch.float)
            
            inputii=make_context_vector(a,word2idx)
            targett=make_context_vector1(b,word2idx)
            a,b=model(inputii,targett)
            
            a=a.type(torch.float)
            b=b.type(torch.float)
            #print(torch.sum(torch.mul(a,b),dim=1))
            
            
            #loss=loss + torch.sum(torch.mul(a,b),dim=1)
            loss=loss+nn.BCEWithLogitsLoss()(a,b)
        loss.backward()
        optimiser.step()
        print(loss)
        
        
    
    
    
    
    
    
    
    
    


tensor([[0.0072, 0.0076, 0.0071, 0.0074, 0.0072, 0.0068, 0.0073, 0.0078, 0.0079,
         0.0079, 0.0074, 0.0078, 0.0076, 0.0079, 0.0078, 0.0076, 0.0073, 0.0069,
         0.0068, 0.0075, 0.0068, 0.0068, 0.0076, 0.0068, 0.0066, 0.0072, 0.0070,
         0.0070, 0.0071, 0.0070, 0.0069, 0.0072, 0.0075, 0.0069, 0.0066, 0.0066,
         0.0076, 0.0076, 0.0072, 0.0070, 0.0076, 0.0078, 0.0073, 0.0069, 0.0076,
         0.0078, 0.0077, 0.0077, 0.0079, 0.0078, 0.0072, 0.0074, 0.0078, 0.0075,
         0.0077, 0.0073, 0.0067, 0.0079, 0.0075, 0.0074, 0.0072, 0.0067, 0.0074,
         0.0075, 0.0078, 0.0069, 0.0079, 0.0068, 0.0071, 0.0071, 0.0068, 0.0081,
         0.0075, 0.0072, 0.0067, 0.0070, 0.0075, 0.0076, 0.0070, 0.0068, 0.0070,
         0.0064, 0.0074, 0.0067, 0.0079, 0.0067, 0.0074, 0.0075, 0.0072, 0.0072,
         0.0067, 0.0076, 0.0079, 0.0068, 0.0067, 0.0077, 0.0070, 0.0075, 0.0079,
         0.0075, 0.0074, 0.0077, 0.0076, 0.0071, 0.0076, 0.0069, 0.0074, 0.0073,
         0.0067, 0.0069, 0.0

tensor([138.3260], grad_fn=<AddBackward0>)
tensor([138.3242], grad_fn=<AddBackward0>)
tensor([138.3190], grad_fn=<AddBackward0>)
tensor([138.3177], grad_fn=<AddBackward0>)
tensor([138.3175], grad_fn=<AddBackward0>)
tensor([138.3174], grad_fn=<AddBackward0>)
tensor([138.3172], grad_fn=<AddBackward0>)
tensor([138.3172], grad_fn=<AddBackward0>)
tensor([138.3170], grad_fn=<AddBackward0>)
tensor([138.3170], grad_fn=<AddBackward0>)
tensor([138.3169], grad_fn=<AddBackward0>)
tensor([138.3168], grad_fn=<AddBackward0>)
tensor([138.3156], grad_fn=<AddBackward0>)
tensor([138.3106], grad_fn=<AddBackward0>)
tensor([138.3111], grad_fn=<AddBackward0>)
tensor([138.3106], grad_fn=<AddBackward0>)
tensor([138.3106], grad_fn=<AddBackward0>)
tensor([138.3105], grad_fn=<AddBackward0>)
tensor([138.3104], grad_fn=<AddBackward0>)
tensor([138.3102], grad_fn=<AddBackward0>)
tensor([138.3048], grad_fn=<AddBackward0>)
tensor([138.3042], grad_fn=<AddBackward0>)
tensor([138.3040], grad_fn=<AddBackward0>)
tensor([138

tensor([138.2384], grad_fn=<AddBackward0>)
tensor([138.2384], grad_fn=<AddBackward0>)
tensor([138.2384], grad_fn=<AddBackward0>)
tensor([138.2364], grad_fn=<AddBackward0>)
tensor([138.2327], grad_fn=<AddBackward0>)
tensor([138.2328], grad_fn=<AddBackward0>)
tensor([138.2327], grad_fn=<AddBackward0>)
tensor([138.2329], grad_fn=<AddBackward0>)
tensor([138.2327], grad_fn=<AddBackward0>)
tensor([138.2328], grad_fn=<AddBackward0>)
tensor([138.2326], grad_fn=<AddBackward0>)
tensor([138.2327], grad_fn=<AddBackward0>)
tensor([138.2287], grad_fn=<AddBackward0>)
tensor([138.2267], grad_fn=<AddBackward0>)
tensor([138.2259], grad_fn=<AddBackward0>)
tensor([138.2263], grad_fn=<AddBackward0>)
tensor([138.2261], grad_fn=<AddBackward0>)
tensor([138.2261], grad_fn=<AddBackward0>)
tensor([138.2260], grad_fn=<AddBackward0>)
tensor([138.2260], grad_fn=<AddBackward0>)
tensor([138.2261], grad_fn=<AddBackward0>)
tensor([138.2259], grad_fn=<AddBackward0>)
tensor([138.2259], grad_fn=<AddBackward0>)
tensor([138

tensor([138.2321], grad_fn=<AddBackward0>)
tensor([138.2319], grad_fn=<AddBackward0>)
tensor([138.2318], grad_fn=<AddBackward0>)
tensor([138.2311], grad_fn=<AddBackward0>)
tensor([138.2317], grad_fn=<AddBackward0>)
tensor([138.2315], grad_fn=<AddBackward0>)
tensor([138.2311], grad_fn=<AddBackward0>)
tensor([138.2313], grad_fn=<AddBackward0>)
tensor([138.2311], grad_fn=<AddBackward0>)
tensor([138.2311], grad_fn=<AddBackward0>)
tensor([138.2307], grad_fn=<AddBackward0>)
tensor([138.2307], grad_fn=<AddBackward0>)
tensor([138.2309], grad_fn=<AddBackward0>)
tensor([138.2307], grad_fn=<AddBackward0>)
tensor([138.2308], grad_fn=<AddBackward0>)
tensor([138.2306], grad_fn=<AddBackward0>)
tensor([138.2308], grad_fn=<AddBackward0>)
tensor([138.2307], grad_fn=<AddBackward0>)
tensor([138.2306], grad_fn=<AddBackward0>)
tensor([138.2301], grad_fn=<AddBackward0>)
tensor([138.2303], grad_fn=<AddBackward0>)
tensor([138.2301], grad_fn=<AddBackward0>)
tensor([138.2300], grad_fn=<AddBackward0>)
tensor([138

In [63]:
#print(data[3][0])

with torch.no_grad():
    inputii=make_context_vector(['the', 'prince', 'with', 'her'],word2idx)
    target=make_context_vector1(data[49][1],word2idx)
    #print(target)
    print(data[49][1])
    a,b=model(inputii,target)
    #print(a.size())
    #print(b.size())
    #print(data[1][0])
    _,c=torch.max(a,1)
    _,d=torch.max(b,1)
    print(c,d)
    
    print(idx2word[52])
    print(idx2word[37])
    

effect,
tensor([52]) tensor([37])
and
effect,


In [53]:
with torch.no_grad():
    inputii=make_context_vector(data[0][0],word2idx)
    target=make_context_vector1(data[0][1],word2idx)
    print(target)
    #loss=model(inputii,target)
    #print(loss)
    
    
targett=one_hot = torch.nn.functional.one_hot(inputii, max_n)
#print(targett)      

tensor(43)
